# Config

In [2]:
from plotly.offline import init_notebook_mode, iplot # from plotly.offline import init_notebook_mode, iplot # 
from plotly.graph_objs import Scatter, Figure, Layout
init_notebook_mode(connected=True)

In [3]:
import numpy as np
from scipy.integrate import odeint

# Simplest Plot

In [9]:
x = list(range(0,10))
y = [c*c for c in x]
# iplot([{"x": x, "y": y}])

# Dynamics (Equations of Motion)

$m\ddot{x} + b \dot{x} + kx = u$

$y = \begin{bmatrix}x \\ \dot{x}\end{bmatrix}$

In [6]:
# Equations of motion
def get_x_dotdot(x, x_dot, u):
    m = 10
    b = 1*0
    k = 1
    x_dotdot = (u-k*x-b*x_dot)/m
    return x_dotdot

# ODE
def get_y_dot(y,t,u):
    x, x_dot = y
    x_dotdot = get_x_dotdot(x,x_dot, u)
    y_dot = [x_dot, x_dotdot]
    return y_dot
    
# PD controller    
def get_u(x,x_dot):
    kp = 10
    kd = 6
    u = -kp*x-kd*x_dot
    return u    

# Simulation

Dynamics in continous time; controller in discrete time (zero order hold)

In [10]:
# Initial Conditions
y0 = [0, 3]

# Initialize empty result vectors
t_all = np.empty(shape=[0, 1]) 
y_all = np.empty(shape=[0, 2]) 
end = -1

# Time settings
dt    = 0.5 # Controller sampling time
t_end = 20  # Simulation time
Tdis = np.arange(0,t_end,dt) # Discrete time samples

# At every sample do
for tnow in Tdis:
    
    # Given the current state, compute a control signal
    x,x_dot = y0
    u = get_u(x,x_dot)
    
    # Integrate equation of motion until next sample while holding control
    simsteps_per_sample = 100
    t = np.linspace(tnow,tnow+dt,simsteps_per_sample)
    y = odeint(get_y_dot, y0, t, args=(u,))
    
    # The final y state is the initial state for next time
    y0 = y[end,:]
    
    # Append the results
    t_all = np.append(t_all,t)
    y_all = np.append(y_all,y, axis = 0)
    
    
# Match time and output signals for plotting    
pos_plot = Scatter(x = t_all, y = y_all[:, 0], name = 'position')   
vel_plot = Scatter(x = t_all, y = y_all[:, 1], name = 'speed')   
    
# Plot results inline    
iplot([pos_plot,vel_plot])   